In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
file = '0.npy'
item_with_index = np.load(file, allow_pickle=True)
item = item_with_index[0].astype(np.float16)
max_len = 2537
def add_padding(max_frames, item):
    current_length = item.shape[0]
    if current_length < max_frames:
        padding = np.full(
            (max_frames - current_length, 2172), 
            -1,
            dtype=np.float16
        )
        padding[:, 3::4] = 0
        item = np.concatenate((item, padding), axis=0)
    return item
matrix = add_padding(max_len, item)
print(matrix.shape)

In [3]:
# Paso 1: Generar la matriz y centrar los datos restando la media de cada columna
matrix_meaned = matrix - np.mean(matrix, axis=0)

# Paso 2: Calcular la matriz de covarianza
cov_matrix = np.cov(matrix_meaned, rowvar=False)

# Paso 3: Calcular los valores y vectores propios
eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

# Paso 4: Filtrar valores propios negativos o muy cercanos a cero
eigenvalues = eigenvalues[eigenvalues > 1e-10]  # Filtramos los valores muy pequeños o negativos

# Ordenar los valores propios en orden descendente
sorted_eigenvalues = np.sort(eigenvalues)[::-1]

In [ ]:
sorted_eigenvalues

In [ ]:
# Paso 5: Graficar los valores propios en un gráfico de barras verticales (bar)
plt.figure(figsize=(10, 8))
plt.bar(range(len(sorted_eigenvalues)), sorted_eigenvalues, color='blue')
plt.xlabel('Component Number')
plt.ylabel('Eigenvalue')
plt.title('PCA Eigenvalues Ordered by Importance')

plt.show()

In [ ]:
pc = PCA()
pc.fit(matrix)

In [ ]:
plt.figure(figsize=(10, 5))
plt.title('Principal Component Analysis')
plt.plot(pc.explained_variance_ratio_)
print(pc.explained_variance_ratio_)
plt.legend(['Explained Variance'])
plt.xlabel('N Components')
plt.ylabel('Explained Variance Ratio')
plt.show()

In [ ]:
pc = PCA(n_components=1)
new_matrix = pc.fit_transform(matrix)
print("Original matrix shape:", matrix.shape)
print("New matrix shape after PCA:", new_matrix.shape)

In [ ]:
# Paso 2: Graficar la varianza explicada por cada componente principal
plt.figure(figsize=(10, 5))
plt.bar(range(1, len(pc.explained_variance_ratio_) + 1), pc.explained_variance_ratio_, color='blue')
print(pc.explained_variance_ratio_)
plt.title('Explained Variance Ratio by Principal Component')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.grid(True)
plt.show()

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ReduceMatrixModel(nn.Module):
    def __init__(self):
        super(ReduceMatrixModel, self).__init__()
        
        # First Convolutional Layer: Reduce size from (2537, 2172)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)
        
        # Adjust based on the final conv layer output size (256 * 159 * 136 = 5535744)
        self.fc1 = nn.Linear(256 * 159 * 136, 512)  
        
    def forward(self, x):
        print(x.shape) # -> (batch_size, 1, 2537, 2172)
        x = F.relu(self.conv1(x))  # -> (batch_size, 32, 1269, 1086)
        print(x.shape)
        x = F.relu(self.conv2(x))  # -> (batch_size, 64, 635, 543)
        print(x.shape)
        x = F.relu(self.conv3(x))  # -> (batch_size, 128, 318, 272)
        print(x.shape)
        x = F.relu(self.conv4(x))  # -> (batch_size, 256, 159, 136)
        print(x.shape)

        x = x.view(x.size(0), -1)  # -> (batch_size, 256 * 159 * 136 = 5535744)
        print(x.shape)

        # Fully connected layers
        x = F.relu(self.fc1(x))  # -> (batch_size, 512)
        return x

# Example of model usage
model = ReduceMatrixModel()


In [ ]:
matrix = np.random.rand(2537, 2172)
input_matrix = torch.tensor([[matrix]]).float()
output = model(input_matrix).squeeze()
print(output.shape)  # Should print torch.Size([1, 512])

In [ ]:
print(output)